# Testing merges before turning them into functions

In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
organizations= data_prep.load_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Merge NTD with GTFS  --> m2_df Function
* Delete 'legacy ntd id", primary_uza_pop, reporter_type, & Years old in vehicles data set?

In [3]:
#reading in crosswalk 1
crosswalk1 = pd.read_parquet('test_crosswalk_both.parquet', engine='pyarrow')

In [4]:
#reading crosswalk 2 (the manual one that captures some of the missing values)
crosswalk2 = pd.read_csv('vehicles-left-only.csv')

In [5]:
#rename to agency so we can match it
crosswalk1 = crosswalk1.rename(columns = {'agency_x':'agency'})

In [6]:
#first crosswalk
m1_df = vehicles.merge(crosswalk1, on=['agency', 'ntd_id'],  how='left', indicator=True)

In [7]:
m1_df = m1_df.drop(columns = ['_merge'])

In [8]:
#second crosswalk
m2_df = m1_df.merge(crosswalk2, on=['agency', 'ntd_id', 'itp_id'],  how='left', indicator=True)
m2_df = m2_df.drop(columns = ['_merge'])

In [9]:
#merging the 2 datasets together
vehicles_gtfs = m2_df.merge(organizations, on=['agency', 'ntd_id', 'itp_id'],  how='left', indicator=True)

In [10]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

# Merging Black Cat with vehicles_gtfs

In [99]:
#trying to merge in crosswalk
test1= df_5311.merge(crosswalk3, on=['organization_name'],  how='left', indicator=True)

In [103]:
test1.head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,_merge
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.0,0.0,0,Open,NaN,NaN,left_only


In [17]:
right_on = ["agency"]
left_on = ["organization_name"]

In [18]:
m4_df = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [19]:
#stuff to filter out before cutting off at scores...
subset3 = ['Plumas County Transportation Commission', 'Imperial County Transportation Commission', 'Glenn County Transportation Commission','Tehama County Transit Agency','Tuolumne County Transit Agency (TCTA)']

In [20]:
#filter out a bunch of stuff all at once
m4_df = m4_df[~m4_df.organization_name.isin(subset3)]

In [21]:
#filter out for any scores below 0.075
m5_df = m4_df.loc[m4_df['best_match_score'] > 0.075 ]

In [22]:
m5_df['organization_name'].nunique()

43

In [64]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('BlackCat_NTD_CalITP.xlsx')

In [92]:
black_cat_merged =  m5_df.merge(crosswalk3, on=['organization_name','agency','ntd_id'],  how='left')

In [42]:
black_cat_merged.head(1)

,best_match_score,__id_left,__id_right,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,itp_id,gtfs_schedule_status,#_services_w__complete_rt_status,#_fixed_route_services_w__static_gtfs,complete_static_gtfs_coverage__1=yes_,complete_rt_coverage,>=1_gtfs_feed_for_any_service__1=yes_,>=_1_complete_rt_set__1=yes_
0,0.276762,0_left,132_right,2011.0,Section 5311,CA-18-X047 | 0012000083,2016.0,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.0,0.0,0.0,Open,"City of Chowchilla, dba: Chowchilla Area Transit",Chowchilla,CA,9R02-058,9R02-91071,"City, County or Local Government Unit or Depar...",Rural Reporter,0.0,2.0,Cutaway,NaN,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.75,0.0,4.0,0.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
